## Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
%cd '/content/gdrive/MyDrive/Deep Learning/SiameseNets'

/content/gdrive/MyDrive/Deep Learning/SiameseNets


In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop

## Defining utility functions

In [ ]:
# Function to read .pgm image
def read_image(filename, byteorder='>'):
    #first we read the image, as a raw file to the buffer
    with open(filename, 'rb') as f:
        buffer = f.read()
    
    #using regex, we extract the header, width, height and maxval of the image
    header, width, height, maxval = re.search(
        b"(^P5\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    
    #then we convert the image to numpy array using np.frombuffer which interprets buffer as one dimensional array
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))

In [ ]:
read_image('./Dataset/s1/1.pgm')

array([[48, 49, 45, ..., 56, 56, 54],
       [45, 52, 39, ..., 52, 50, 51],
       [45, 50, 42, ..., 48, 53, 50],
       ...,
       [50, 48, 50, ..., 45, 46, 46],
       [45, 54, 49, ..., 46, 47, 47],
       [51, 51, 51, ..., 47, 46, 46]], dtype=uint8)

In [ ]:
size = 2
total_sample_size = 10000

# Function to generate similar and dissimilar image pairs from the dataset
def get_data(size, total_sample_size):
    #read the image
    image = read_image('./Dataset/s' + str(1) + '/' + str(1) + '.pgm', 'rw+')
    #reduce the size
    image = image[::size, ::size]
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0
    
    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2]) # 2 is for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    
    for i in range(40):
        for j in range(int(total_sample_size/40)):
            ind1 = 0
            ind2 = 0
            
            #read images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(10)
                ind2 = np.random.randint(10)
            
            # read the two images
            img1 = read_image('./Dataset/s' + str(i+1) + '/' + str(ind1 + 1) + '.pgm', 'rw+')
            img2 = read_image('./Dataset/s' + str(i+1) + '/' + str(ind2 + 1) + '.pgm', 'rw+')
            
            #reduce the size
            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]
            
            #store the images to the initialized numpy array
            x_geuine_pair[count, 0, 0, :, :] = img1
            x_geuine_pair[count, 1, 0, :, :] = img2
            
            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
    y_imposite = np.zeros([total_sample_size, 1])
    
    for i in range(int(total_sample_size/10)):
        for j in range(10):
            
            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(40)
                ind2 = np.random.randint(40)
                if ind1 != ind2:
                    break
                    
            img1 = read_image('./Dataset/s' + str(ind1+1) + '/' + str(j + 1) + '.pgm', 'rw+')
            img2 = read_image('./Dataset/s' + str(ind2+1) + '/' + str(j + 1) + '.pgm', 'rw+')

            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]

            x_imposite_pair[count, 0, 0, :, :] = img1
            x_imposite_pair[count, 1, 0, :, :] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1
            
    #now, concatenate, genuine pairs and opposite pair to get the whole data
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y

In [ ]:
X, Y = get_data(size, total_sample_size)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

## Implementation of Siamese Network

In [ ]:
def build_base_network(input_shape):
    
    seq = Sequential()
    
    nb_filter = [6, 12]
    kernel_size = 3
    
    
    #convolutional layer 1
    seq.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=input_shape,
                          padding='valid', data_format="channels_first"))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first")) 
    seq.add(Dropout(.25))
    
    #convolutional layer 2
    seq.add(Convolution2D(nb_filter[1], kernel_size, kernel_size, padding='valid', data_format="channels_first"))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first")) 
    seq.add(Dropout(.25))

    #flatten 
    seq.add(Flatten())
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(50, activation='relu'))
    return seq

In [ ]:
input_dim = x_train.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [ ]:
epochs = 30
rms = RMSprop()

model = Model(inputs=[img_a, img_b], outputs=distance)

In [ ]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

model.compile(loss=contrastive_loss, optimizer=rms)

## Training and testing

In [ ]:
img_1 = x_train[:, 0]
img_2 = x_train[:, 1] 
print(img_1.shape, img_2.shape)
model.fit([img_1, img_2], y_train, validation_split=.25, batch_size=128, verbose=2, epochs=epochs)

(15000, 1, 56, 46) (15000, 1, 56, 46)
Epoch 1/30
88/88 - 2s - loss: 0.1893 - val_loss: 0.1982 - 2s/epoch - 24ms/step
Epoch 2/30
88/88 - 1s - loss: 0.1864 - val_loss: 0.1881 - 749ms/epoch - 9ms/step
Epoch 3/30
88/88 - 1s - loss: 0.1836 - val_loss: 0.1870 - 712ms/epoch - 8ms/step
Epoch 4/30
88/88 - 1s - loss: 0.1774 - val_loss: 0.1759 - 720ms/epoch - 8ms/step
Epoch 5/30
88/88 - 1s - loss: 0.1762 - val_loss: 0.1688 - 712ms/epoch - 8ms/step
Epoch 6/30
88/88 - 1s - loss: 0.1746 - val_loss: 0.1804 - 728ms/epoch - 8ms/step
Epoch 7/30
88/88 - 1s - loss: 0.1734 - val_loss: 0.1663 - 731ms/epoch - 8ms/step
Epoch 8/30
88/88 - 1s - loss: 0.1719 - val_loss: 0.1658 - 712ms/epoch - 8ms/step
Epoch 9/30
88/88 - 1s - loss: 0.1688 - val_loss: 0.1552 - 706ms/epoch - 8ms/step
Epoch 10/30
88/88 - 1s - loss: 0.1667 - val_loss: 0.1522 - 699ms/epoch - 8ms/step
Epoch 11/30
88/88 - 1s - loss: 0.1660 - val_loss: 0.1547 - 688ms/epoch - 8ms/step
Epoch 12/30
88/88 - 1s - loss: 0.1647 - val_loss: 0.1490 - 750ms/epoch 

In [ ]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])

In [ ]:
def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

In [ ]:
compute_accuracy(pred, y_test)

0.7357366771159874